In [27]:
import csv
import pandas as pd
import numpy as np
import os
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
import random
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_log_error as MSLE
random.seed(123)
plt.rcParams["font.sans-serif"]=["SimHei"]
from datetime import datetime
pd.set_option("display.max_rows", 1000)    #設定最大能顯示1000rows
pd.set_option("display.max_columns", 1000) #設定最大能顯示1000columns
path = r"C:\Users\user\Desktop\ETC"

In [28]:
acc_1_fin=pd.read_csv(path+r"/acc_1_fin.csv")
print(acc_1_fin.columns)
print(acc_1_fin.shape)

Index(['Unnamed: 0', '回堵情況', '總通行量', '總平均車速', '里程數', '速限', '24小時內死亡人數',
       '2-30日內死亡人數', '受傷人數', '事故涉及車輛數',
       ...
       '肇因研判(子)_22', '肇因研判(子)_23', '肇因研判(子)_24', '肇因研判(子)_25', '肇因研判(子)_26',
       '肇因研判(子)_27', '肇因研判(子)_28', '是否為連續假日_1', '是否為假日_1', '處理時間'],
      dtype='object', length=281)
(7252, 281)


In [29]:
X1 = acc_1_fin.drop("處理時間",axis = 1)
y1 = acc_1_fin["處理時間"]

In [30]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2) # 0.2 test_size means 20%

In [31]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

# GridSearchCV()

In [32]:
dt_gsc = GridSearchCV(DecisionTreeRegressor(random_state=0),
              param_grid={
                  'min_samples_split': range(2, 10),
                  'max_depth': range(3,7)},
              scoring='neg_mean_squared_error', verbose=0,n_jobs=-1, cv=10, refit=True)
dt_gsc.fit(X1_train,y1_train)
dt_gsc.best_params_

{'max_depth': 4, 'min_samples_split': 9}

In [33]:
dt = DecisionTreeRegressor(random_state=0, criterion="mse",max_depth=dt_gsc.best_params_["max_depth"],
                           min_samples_split=dt_gsc.best_params_["min_samples_split"])
dt_scores = cross_val_score(dt, X1_train, y1_train, cv=10,scoring='neg_mean_squared_error')

In [34]:
dt_fit = dt.fit(X1_train,y1_train)

In [35]:
## predict_y
de_tree_1_pred=dt.predict(X1_test)
de_tree_1_pred=de_tree_1_pred.flatten()

In [36]:
# msle
print(MSLE(de_tree_1_pred,y1_test,))
# rmse
print(np.sqrt(mean_squared_error(y1_test,de_tree_1_pred)))
# mse
print(mean_squared_error(y1_test,de_tree_1_pred))
# r2
print(metrics.r2_score(y1_test,de_tree_1_pred))

0.4716388696628134
12.689079434643896
161.01273689870266
0.18371539285473293
